<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#加载数据和模型" data-toc-modified-id="加载数据和模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>加载数据和模型</a></span></li><li><span><a href="#精度，召回率，F和时间随参数的变化" data-toc-modified-id="精度，召回率，F和时间随参数的变化-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>精度，召回率，F和时间随参数的变化</a></span></li></ul></div>

In [1]:
%matplotlib inline
dataset = '03_brightness'
raw_path = f'/home/liucc/Workspace/02_Ellipse/03_results/{dataset}/01_raw/'
result_path = f'/home/liucc/Workspace/02_Ellipse/03_results/{dataset}/05_ellipse_result/proposed/'

In [2]:
import os 
fnames = os.listdir(raw_path)
fnames = [n[:-4] for n in fnames if n[-3:]=='png']
fnames.sort()
print(fnames)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040']


# 加载数据和模型

In [3]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/01_data')
from cc_process_image import *
import imageio

raws = get_all_images(f'{raw_path}',fnames,'png')

print(len(raws))

40


In [4]:
sys.path.append('/home/liucc/Workspace/02_Ellipse/02_algorithm/01_cnn')
from cnn_model import *

model_path = r'/home/liucc/Workspace/02_Ellipse/06_model/'
model_name = r'ellipse_brightness'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = RestoreNet().to(device)

model.load_state_dict(torch.load(f'{model_path}{model_name}.mdl')['state_dict'])

# 精度，召回率，F和时间随参数的变化

In [5]:
import sys
sys.path.append('/home/liucc/Workspace/02_Ellipse/02_algorithm/02_ellipse_detect')
from cc_ellipse import *
sys.path.append('../')
from experiment import *

In [6]:
import time
from skimage.feature import canny
#全局参数
GLOB = {
    'MIN_LEN' : 60,
    'MIN_COVER_ANGLE' : 80,
    'MIN_COVER_RHO1' : 0.5,
    'MIN_COVER_RHO2' : 0.2,
}

def one_task(glob):
    start = time.time()

    prs = []
    
    pre_time = 0
    detect_time = 0
    
    for name,im in zip(fnames,raws):
        oname = result_path+name+'.txt'
        
        start = time.time()
        im = preprocess_image(im,scale=True)
        imo = predict(model,im, device)
        end = time.time()
        
        pre_time+= end-start
        
        start = time.time()
        imo = postprocess_image(imo,0.3)
        ells = detect_ellipses_im(imo,glob)
        end = time.time()
        
        detect_time+= end-start
        
        prs.append(ells)
        
        with open(oname,'w') as f:
            estr = '\n'.join([' '.join(['%.4f'%v for v in l]) for l in ells])
            f.write(estr)
    
    
    pre_time = pre_time/len(raws)
    detect_time = detect_time/len(raws)
    totat_time = pre_time+detect_time
    
    return [pre_time,detect_time,totat_time]

In [7]:
pre_time,detect_time,totat_time = one_task(GLOB)
print('Process Time: %.0f' % (pre_time*1000))
print('Detect Time: %.0f' % (detect_time*1000))
print('Total Time: %.0f' % (totat_time*1000))

Process Time: 46
Detect Time: 104
Total Time: 150
